All needed packages are listed in `requirements.txt`, meaning that missing packages will be downloaded.

In [ ]:
# Install needed packages

!pip install -r requirements.txt

In [ ]:
def check_dirs(folder_name):
    dir_name = folder_name+'_imgs'
    if os.path.exists(dir_name):
        print('[INFO] ' + folder_name + ' directories already existed...')
        pass
    else:
        os.mkdir(dir_name)
        os.mkdir(dir_name+'/ok')
        os.mkdir(dir_name+'/ng')
        print('[INFO] '+ folder_name + ' directories have been created...')

check_dirs('resize')


In [ ]:
# Necessary import statements
import Augmentor
import cv2
import os

# Set variables and directories
DATADIR = "Project_Images"      # Directoy where images were inited stored
RESIZE_DIR = "resize_imgs"     # Directory where resized images will be stored
CATEGORIES = ["ng", "ok"]       
IMG_SIZE = 299                  # Standard resolution
training_data = []

# Functions

# Define Region of interest
def roi(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
                
    # Find contour and sort by contour area
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # Find bounding box and extract ROI
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = image[y:y+h, x:x+w]
        break                
    new_array = cv2.resize(ROI, (IMG_SIZE, IMG_SIZE),interpolation = cv2.INTER_AREA)
    return new_array

# Read img, crop and save to new path
def save_cropped_image():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))  # read image
                
                new_array = roi(img_array)
                new_path = RESIZE_DIR+"/"+category
                new_img = "ROI_"+img
                resize_path = os.path.join(new_path, new_img)
                cv2.imwrite(resize_path, new_array)
                training_data.append([new_array, class_num])
                print("[INFO] New image has been saved: " + new_img)
                print("[INFO] "+ new_img + " has been added to training data")
            except Exception as e:
                pass

# Implement data agumentation
def data_augmentation():
    for category in CATEGORIES:
        path = Augmentor.Pipeline(os.path.join(RESIZE_DIR, category))
        # The probability parameter controls how often the operation is applied.
        path.rotate_random_90(probability=0.6)
        path.random_brightness(probability=0.6,min_factor=0.2,max_factor=1.5)
        ## duda - es necesario?
        # path.skew(probability=0.4,magnitude=0.5)
        ##
        print("[INFO] Data augmentation was applied to "+ category)
        path.sample(500)

def add_augmented_imgs_to_training_data():
    print("[INFO] Augmented images will be added to training data")
    for category in CATEGORIES:
        path = os.path.join(RESIZE_DIR, category)
        class_num = CATEGORIES.index(category)
        output_path = os.path.join(path,'output')
        #print("output_path: "+output_path)
        for img in os.listdir(output_path):
            try:
                img_array = cv2.imread(os.path.join(output_path,img), cv2.IMREAD_GRAYSCALE)   # read image
                training_data.append([img_array, class_num])
                print("[INFO] "+img + "has been added to training data")
            except Exception as e:
                pass

# Process to create training data
save_cropped_image()
data_augmentation()
add_augmented_imgs_to_training_data()
print("Training data length: "+str(len(training_data)))

In [21]:
import random
random.shuffle(training_data)

In [27]:
import numpy as np
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

C:\Users\espin\AppData\Local\Temp/ipykernel_19948/1413217270.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)


ValueError: could not broadcast input array from shape (299,299,3) into shape (299,299)

In [5]:
import pickle
pickle_out = open("X_2.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y_2.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()